Mining Wikipedia Election Data
=========================

This notebook mines the data in the Wikipedia pages for the following elections:

- 2016 United States presidential election
- 2016 United States Senate elections
- 2016 United States House of Representatives elections
- 2017 United States Senate elections (special elections)
- 2018 United States Senate elections
- 2018 United States House of Representatives elections

Timestamp: 12:00 PM ET, 11 Aug. 2020

In [1]:
# !pip3 install pandas
# !pip3 install wikipedia

# !pip3 install git+https://github.com/KeiferC/gdutils.git

In [2]:
import pandas as pd
import numpy as np
import wikipedia
import os
import re

import gdutils.datamine as dm
import gdutils.dataqa as dq
import gdutils.extract as et

from typing import Any, List, Tuple, Dict, Hashable, Optional, Union, NoReturn

In [3]:
state_names = [
    'Alabama', 'Alaska','Arizona', 'Arkansas', 'California', 
    'Colorado', 'Connecticut', 'Delaware',  'Florida', 'Georgia', 
    'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 
    'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 
    'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 
    'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 
    'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 
    'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 
    'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 
    'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

state_abbreviations = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 
    'CO', 'CT', 'DE', 'FL', 'GA', 
    'HI', 'ID', 'IL', 'IN', 'IA', 
    'KS', 'KY', 'LA', 'ME', 'MD', 
    'MA', 'MI', 'MN', 'MS', 'MO', 
    'MT', 'NE', 'NV', 'NH', 'NJ', 
    'NM', 'NY', 'NC', 'ND', 'OH', 
    'OK', 'OR', 'PA', 'RI', 'SC', 
    'SD', 'TN', 'TX', 'UT', 'VT', 
    'VA', 'WA', 'WV', 'WI', 'WY']

states = list(zip(state_names, state_abbreviations))

Step 1. Generate Wikipedia page titles for scraping
------------------------------------------------------------

In [4]:
pres_election = ('PRES', 'United States presidential election')

fed_elections = [('SEN',  'United States Senate election'),
                 ('USH',  'United States House of Representatives election')]

election_years_to_check = [2016, 2017, 2018]

In [5]:
wiki_titles = []

for yr in election_years_to_check:
    generate_key = lambda yr, ekey, st_abv: ekey + str(yr % 100) + '_' + st_abv
    generate_title = lambda yr, etype, st: str(yr) + ' ' + etype + ' in ' + st
    
    if yr % 4 == 0:
        [wiki_titles.append((generate_key(yr, pres_election[0], st_abv),
                             generate_title(yr, pres_election[1], st)))
         for st, st_abv in states]
        
    [wiki_titles.append((generate_key(yr, ekey, st_abv),
                         generate_title(yr, etype, st)))
     for ekey, etype in fed_elections
     for st, st_abv in states]

Step 2. Gather Wikipedia URLs from page titles
------------------------------------------------------------

In [6]:
# wiki_urls = {}

# for wiki_title in wiki_titles:
#     key, title = wiki_title
    
#     try:
#         url = wikipedia.page(title=title).url

#         if set(title.split(' ')).issubset(
#                 set(re.findall('[a-zA-Z0-9]+', url))):
#             wiki_urls[key] = (title, url)
            
#     except Exception:
#         continue # it's okay if page does not exist

In [7]:
# Print retrieved page URLs
# Necessary for manually verifying URL-to-election mapping since
# Wikipedia API tries to find best match, not the exact match

# for wiki_key in wiki_urls:
#     title, url = wiki_urls[wiki_key]
#     print('{:9} : {}\n\t{}'.format(wiki_key, title, url))

Step 3. Gather tabular data from Wikipedia pages
------------------------------------------------------------

In [8]:
# wiki_tables = {}

# for wiki_key in wiki_urls:
#     try:
#         wiki_tables[wiki_key] = pd.read_html(wiki_urls[wiki_key][1])
#     except Exception as e:
#         print("Unable to gather Wikipedia tabular data:", e)

In [9]:
# Display wikipedia tabular election data
# Necessary for finding applicable table because a page can 
# contain multiple nameless tables whose orders differ from
# other pages

def print_wiki_tables(key):
    for wiki in wiki_tables:
        if wiki.startswith(key):
            print('================================================')
            print('Wiki: {} '.format(wiki))
            print('================================================')

            for i in range(len(wiki_tables[wiki])):
                print('TABLE {}: ############################\n{}\n\n\n'.format(
                        i, wiki_tables[wiki][i].head()))

In [10]:
# commented out to save screen space
# print_wiki_tables('PRES16')

In [11]:
# commented out to save screen space
# print_wiki_tables('SEN16')

In [12]:
# commented out to save screen space
# print_wiki_tables('USH16')

In [13]:
# commented out to save screen space
# print_wiki_tables('SEN17')

In [14]:
# commented out to save screen space
# print_wiki_tables('USH17') # no data exists

In [15]:
# commented out to save screen space
# print_wiki_tables('SEN18')

In [16]:
# commented out to save screen space
# print_wiki_tables('USH18')

Step 4. Collect applicable election data from tables
------------------------------------------------------------

In [17]:
# Has to be done manually because every Wikipedia page 
# is different and because Wikipedia doesn't have datasets
# to download and thus all tables are scraped

# wiki_dfs = {}

__2016 United States presidential election data (``PRES16``)__

In [18]:
# Unless otherwise stated, all data below are at the state-level

# wiki_dfs['PRES16_AL'] = wiki_tables['PRES16_AL'][5]  
# wiki_dfs['PRES16_AK'] = wiki_tables['PRES16_AK'][17]
# wiki_dfs['PRES16_AZ'] = wiki_tables['PRES16_AZ'][19] 
# wiki_dfs['PRES16_AR'] = wiki_tables['PRES16_AR'][7]  
# wiki_dfs['PRES16_CA'] = wiki_tables['PRES16_CA'][29] 
# wiki_dfs['PRES16_CO'] = wiki_tables['PRES16_CO'][20] 
# wiki_dfs['PRES16_CT'] = wiki_tables['PRES16_CT'][15] 
# wiki_dfs['PRES16_DE'] = wiki_tables['PRES16_DE'][12] 
# wiki_dfs['PRES16_FL'] = wiki_tables['PRES16_FL'][13] 
# wiki_dfs['PRES16_GA'] = wiki_tables['PRES16_GA'][12] 
# wiki_dfs['PRES16_HI'] = wiki_tables['PRES16_HI'][11] 
# wiki_dfs['PRES16_ID'] = wiki_tables['PRES16_ID'][15] 
# wiki_dfs['PRES16_IL'] = wiki_tables['PRES16_IL'][24] 
# wiki_dfs['PRES16_IN'] = wiki_tables['PRES16_IN'][14] # county-level
# wiki_dfs['PRES16_IA'] = wiki_tables['PRES16_IA'][12] 
# wiki_dfs['PRES16_KS'] = wiki_tables['PRES16_KS'][15] 
# wiki_dfs['PRES16_KY'] = wiki_tables['PRES16_KY'][12] # county-level 
# wiki_dfs['PRES16_LA'] = wiki_tables['PRES16_LA'][8] 
# wiki_dfs['PRES16_ME'] = wiki_tables['PRES16_ME'][18] 
# wiki_dfs['PRES16_MD'] = wiki_tables['PRES16_MD'][14] 
# wiki_dfs['PRES16_MA'] = wiki_tables['PRES16_MA'][15] 
# wiki_dfs['PRES16_MI'] = wiki_tables['PRES16_MI'][16] 
# wiki_dfs['PRES16_MN'] = wiki_tables['PRES16_MN'][17] 
# wiki_dfs['PRES16_MS'] = wiki_tables['PRES16_MS'][15] 
# wiki_dfs['PRES16_MO'] = wiki_tables['PRES16_MO'][15] 
# wiki_dfs['PRES16_MT'] = wiki_tables['PRES16_MT'][10] # county-level
# wiki_dfs['PRES16_NE'] = wiki_tables['PRES16_NE'][25] 
# wiki_dfs['PRES16_NV'] = wiki_tables['PRES16_NV'][16] 
# wiki_dfs['PRES16_NH'] = wiki_tables['PRES16_NH'][19] 
# wiki_dfs['PRES16_NJ'] = wiki_tables['PRES16_NJ'][13] 
# wiki_dfs['PRES16_NM'] = wiki_tables['PRES16_NM'][12] 
# wiki_dfs['PRES16_NY'] = wiki_tables['PRES16_NY'][26] 
# wiki_dfs['PRES16_NC'] = wiki_tables['PRES16_NC'][17] 
# wiki_dfs['PRES16_ND'] = wiki_tables['PRES16_ND'][11] 
# wiki_dfs['PRES16_OH'] = wiki_tables['PRES16_OH'][22] 
# wiki_dfs['PRES16_OK'] = wiki_tables['PRES16_OK'][18] 
# wiki_dfs['PRES16_OR'] = wiki_tables['PRES16_OR'][20] 
# wiki_dfs['PRES16_PA'] = wiki_tables['PRES16_PA'][17] 
# wiki_dfs['PRES16_RI'] = wiki_tables['PRES16_RI'][11] 
# wiki_dfs['PRES16_SC'] = wiki_tables['PRES16_SC'][14] 
# wiki_dfs['PRES16_SD'] = wiki_tables['PRES16_SD'][10] 
# wiki_dfs['PRES16_TN'] = wiki_tables['PRES16_TN'][11] 
# wiki_dfs['PRES16_TX'] = wiki_tables['PRES16_TX'][28] 
# wiki_dfs['PRES16_UT'] = wiki_tables['PRES16_UT'][12] 
# wiki_dfs['PRES16_VT'] = wiki_tables['PRES16_VT'][14] 
# wiki_dfs['PRES16_VA'] = wiki_tables['PRES16_VA'][19] 
# wiki_dfs['PRES16_WA'] = wiki_tables['PRES16_WA'][12] 
# wiki_dfs['PRES16_WV'] = wiki_tables['PRES16_WV'][9] 
# wiki_dfs['PRES16_WI'] = wiki_tables['PRES16_WI'][14] 
# wiki_dfs['PRES16_WY'] = wiki_tables['PRES16_WY'][11] 

__2016 United States Senate election data (``SEN16``)__

In [19]:
# All election data below are at the State-level

# wiki_dfs['SEN16_AL'] = wiki_tables['SEN16_AL'][19] 
# wiki_dfs['SEN16_AK'] = wiki_tables['SEN16_AK'][20] 
# wiki_dfs['SEN16_AZ'] = wiki_tables['SEN16_AZ'][45]
# wiki_dfs['SEN16_AR'] = wiki_tables['SEN16_AR'][16]
# wiki_dfs['SEN16_CA'] = wiki_tables['SEN16_CA'][53]
# wiki_dfs['SEN16_CO'] = wiki_tables['SEN16_CO'][25]
# wiki_dfs['SEN16_CT'] = wiki_tables['SEN16_CT'][20]
# wiki_dfs['SEN16_FL'] = wiki_tables['SEN16_FL'][64]
# wiki_dfs['SEN16_GA'] = wiki_tables['SEN16_GA'][16]
# wiki_dfs['SEN16_HI'] = wiki_tables['SEN16_HI'][18]
# wiki_dfs['SEN16_ID'] = wiki_tables['SEN16_ID'][15]
# wiki_dfs['SEN16_IL'] = wiki_tables['SEN16_IL'][29]
# wiki_dfs['SEN16_IN'] = wiki_tables['SEN16_IN'][25]
# wiki_dfs['SEN16_IA'] = wiki_tables['SEN16_IA'][20]
# wiki_dfs['SEN16_KS'] = wiki_tables['SEN16_KS'][17]
# wiki_dfs['SEN16_KY'] = wiki_tables['SEN16_KY'][22]
# wiki_dfs['SEN16_LA'] = wiki_tables['SEN16_LA'][24]
# wiki_dfs['SEN16_MD'] = wiki_tables['SEN16_MD'][29]
# wiki_dfs['SEN16_MO'] = wiki_tables['SEN16_MO'][24]
# wiki_dfs['SEN16_NV'] = wiki_tables['SEN16_NV'][32]
# wiki_dfs['SEN16_NH'] = wiki_tables['SEN16_NH'][23]
# wiki_dfs['SEN16_NY'] = wiki_tables['SEN16_NY'][15]
# wiki_dfs['SEN16_NC'] = wiki_tables['SEN16_NC'][42]
# wiki_dfs['SEN16_ND'] = wiki_tables['SEN16_ND'][14]
# wiki_dfs['SEN16_OH'] = wiki_tables['SEN16_OH'][29]
# wiki_dfs['SEN16_OK'] = wiki_tables['SEN16_OK'][12]
# wiki_dfs['SEN16_OR'] = wiki_tables['SEN16_OR'][14]
# wiki_dfs['SEN16_PA'] = wiki_tables['SEN16_PA'][38]
# wiki_dfs['SEN16_SC'] = wiki_tables['SEN16_SC'][16]
# wiki_dfs['SEN16_SD'] = wiki_tables['SEN16_SD'][9]
# wiki_dfs['SEN16_UT'] = wiki_tables['SEN16_UT'][19]
# wiki_dfs['SEN16_VT'] = wiki_tables['SEN16_VT'][12]
# wiki_dfs['SEN16_WA'] = wiki_tables['SEN16_WA'][15]
# wiki_dfs['SEN16_WI'] = wiki_tables['SEN16_WI'][21]

__2016 United States House of Representative election data (``USH16``)__

In [20]:
# All election data below are at the State-level

# wiki_dfs['USH16_AK'] = wiki_tables['USH16_AK'][15]
# wiki_dfs['USH16_DE'] = wiki_tables['USH16_DE'][17]
# wiki_dfs['USH16_MT'] = wiki_tables['USH16_MT'][10]
# wiki_dfs['USH16_ND'] = wiki_tables['USH16_ND'][11]
# wiki_dfs['USH16_SD'] = wiki_tables['USH16_SD'][8]
# wiki_dfs['USH16_VT'] = wiki_tables['USH16_VT'][10]
# wiki_dfs['USH16_WY'] = wiki_tables['USH16_WY'][21]

__2017 United States Senate election data (``SEN17``)__

In [21]:
# All election data below are at the State-level

# wiki_dfs['SEN17_AL'] = wiki_tables['SEN17_AL'][48]

__2017 United States House of Representative election data (``USH17``)__

In [22]:
# No such data exists

__2018 United States Senate election data (``SEN18``)__

In [23]:
# All election data below are at the State-level

# wiki_dfs['SEN18_AZ'] = wiki_tables['SEN18_AZ'][40]
# wiki_dfs['SEN18_CA'] = wiki_tables['SEN18_CA'][54]
# wiki_dfs['SEN18_CT'] = wiki_tables['SEN18_CT'][17] 
# wiki_dfs['SEN18_DE'] = wiki_tables['SEN18_DE'][29]
# wiki_dfs['SEN18_FL'] = wiki_tables['SEN18_FL'][29]
# wiki_dfs['SEN18_HI'] = wiki_tables['SEN18_HI'][14]
# wiki_dfs['SEN18_IN'] = wiki_tables['SEN18_IN'][29]
# wiki_dfs['SEN18_ME'] = wiki_tables['SEN18_ME'][20]
# wiki_dfs['SEN18_MD'] = wiki_tables['SEN18_MD'][25]
# wiki_dfs['SEN18_MA'] = wiki_tables['SEN18_MA'][29]
# wiki_dfs['SEN18_MI'] = wiki_tables['SEN18_MI'][30]
# wiki_dfs['SEN18_MN'] = wiki_tables['SEN18_MN'][20]
# wiki_dfs['SEN18_MS'] = wiki_tables['SEN18_MS'][23]
# wiki_dfs['SEN18_MO'] = wiki_tables['SEN18_MO'][35]
# wiki_dfs['SEN18_MT'] = wiki_tables['SEN18_MT'][22]
# wiki_dfs['SEN18_NE'] = wiki_tables['SEN18_NE'][19]
# wiki_dfs['SEN18_NV'] = wiki_tables['SEN18_NV'][28]
# wiki_dfs['SEN18_NJ'] = wiki_tables['SEN18_NJ'][22]
# wiki_dfs['SEN18_NM'] = wiki_tables['SEN18_NM'][21]
# wiki_dfs['SEN18_NY'] = wiki_tables['SEN18_NY'][16]
# wiki_dfs['SEN18_ND'] = wiki_tables['SEN18_ND'][23]
# wiki_dfs['SEN18_OH'] = wiki_tables['SEN18_OH'][32]
# wiki_dfs['SEN18_PA'] = wiki_tables['SEN18_PA'][28]
# wiki_dfs['SEN18_RI'] = wiki_tables['SEN18_RI'][17]
# wiki_dfs['SEN18_TN'] = wiki_tables['SEN18_TN'][29]
# wiki_dfs['SEN18_TX'] = wiki_tables['SEN18_TX'][37]
# wiki_dfs['SEN18_UT'] = wiki_tables['SEN18_UT'][31]
# wiki_dfs['SEN18_VT'] = wiki_tables['SEN18_VT'][13]
# wiki_dfs['SEN18_VA'] = wiki_tables['SEN18_VA'][32]
# wiki_dfs['SEN18_WA'] = wiki_tables['SEN18_WA'][12]
# wiki_dfs['SEN18_WV'] = wiki_tables['SEN18_WV'][31]
# wiki_dfs['SEN18_WI'] = wiki_tables['SEN18_WI'][27]
# wiki_dfs['SEN18_WY'] = wiki_tables['SEN18_WY'][13]

__2018 United States House of Representative election data (``USH18``)__

In [24]:
# All election data below are at the State-level

# wiki_dfs['USH18_AK'] = wiki_tables['USH18_AK'][14]
# wiki_dfs['USH18_DE'] = wiki_tables['USH18_DE'][19]
# wiki_dfs['USH18_MT'] = wiki_tables['USH18_MT'][13]
# wiki_dfs['USH18_ND'] = wiki_tables['USH18_ND'][15]
# wiki_dfs['USH18_SD'] = wiki_tables['USH18_SD'][11]
# wiki_dfs['USH18_VT'] = wiki_tables['USH18_VT'][12]
# wiki_dfs['USH18_WY'] = wiki_tables['USH18_WY'][10]

Step 5. Save raw scraped tables locally
----------------------------------------------

For future auditing and granular data analysis.

In [25]:
# for election in wiki_dfs:
#     outpath = os.path.join('wiki', 'raw', election + '.csv')
#     et.ExtractTable(wiki_dfs[election], 
#                     outfile=outpath).extract_to_file()

Step 6. Wrangle Wikipedia data
-------------------------------------

In [26]:
party_key_from_candidate = {
    'CLINTON' : 'D',
    'HILLARY' : 'D',
    'RODHAM'  : 'D',

    'JOHNSON' : 'L',
    'GARY'    : 'L',
    
    'STEIN'   : 'G',
    'JILL'    : 'G',
    'Ellen'   : 'G',
    
    'TRUMP'   : 'R',
    'DONALD'  : 'R',
    'JOHN'    : 'R'
}

party_key_from_party = {
    'DEMOCRATIC'    : 'D',
    'DEMOCRAT'      : 'D',
    'DEMOCRATIC-NP' : 'D',
    'GREEN'         : 'G',
    'LIBERTARIAN'   : 'L',
    'REPUBLICAN'    : 'R'
}

candidate_keys = list(party_key_from_candidate.keys())
party_keys = list(party_key_from_party.keys())

In [27]:
wiki_files = dm.list_files_of_type('.csv', os.path.join('wiki', 'raw'))

wiki_dfs = {}
for file in wiki_files:
    wiki_dfs[os.path.basename(file)[:-4]] = pd.read_csv(file)

In [28]:
# returns list of columns in standard upper casing
def upper_cols(df: pd.DataFrame) -> List[str]:
    cols = []
    for col in df.columns:
        try:
            cols.append(col.upper().strip())
        except:
            cols.append(str(col))
            
    return cols
    
# change rows to standard upper casing
def upper_rows():
    for election in wiki_dfs:
        df = wiki_dfs[election]

        for name in df.columns:
            try:
                df[name] = df[name].str.upper()
            except:
                continue

        wiki_dfs[election] = df

In [29]:
for election in wiki_dfs:
    wiki_dfs[election].columns = upper_cols(wiki_dfs[election])

upper_rows()

In [30]:
# drop non-standard data

def col_has_percent(item) -> bool:
    try:
        return '%' in item
    except:
        return False

    
def col_is_empty(item) -> bool:
    return item is None or item is np.nan
    

def col_has_change(item) -> bool:
    try:
        return ('HOLD' in item or 
                'GAIN' in item or
                'WIN'  in item or
                'LOSE' in item)
    except:
        return False
    
    
def drop_cols(dropping_df: pd.DataFrame, cols: List[str], 
              f, threshold: Optional[float] = 0.5) -> pd.DataFrame:
    if not cols:
        return dropping_df

    df = dropping_df.copy()
    filtered = list(df[cols[0]].apply(lambda x: f(x)))
    
    if filtered.count(True) > threshold * len(filtered):
        df = df.drop(columns=cols[0])
    
    cols.pop(0)
    return drop_cols(df, cols, f)

In [31]:
# temp storage of partially processed raw

# for election in wiki_dfs:
#     outpath = os.path.join('wiki', 'temp', election + '.csv')
#     et.ExtractTable(wiki_dfs[election], 
#                     outfile=outpath).extract_to_file()

In [32]:
def get_key_tuple(key: str) -> Tuple[str, str]:
    key_components = key.split('_')
    return (key_components[0], key_components[1])

In [33]:
def get_state_tuple(state_key: str) -> Tuple[str, str]:
    state_tup = [tup for tup in states if tup[1] == state_key]
    state_name, state_abv = state_tup[0]
    return (state_name.upper(), state_abv)

In [34]:
def generate_empty_standardized_df(state_name: str) -> pd.DataFrame:
    standardized_cols = ['STATE', 
                         'PRES16D', 'PRES16G', 'PRES16L', 'PRES16R',
                         'SEN16D',  'SEN16G',  'SEN16L',  'SEN16R',
                         'USH16D',  'USH16G',  'USH16L',  'USH16R',
                         'SEN17D',  'SEN17G',  'SEN17L',  'SEN17R',
                         'USH17D',  'USH17G',  'USH17L',  'USH17R',
                         'SEN18D',  'SEN18G',  'SEN18L',  'SEN18R',
                         'USH18D',  'USH18G',  'USH18L',  'USH18R']
    
    standardized_data = [[state_name,
                         np.nan, np.nan, np.nan, np.nan,
                         np.nan, np.nan, np.nan, np.nan,
                         np.nan, np.nan, np.nan, np.nan,
                         np.nan, np.nan, np.nan, np.nan,
                         np.nan, np.nan, np.nan, np.nan,
                         np.nan, np.nan, np.nan, np.nan,
                         np.nan, np.nan, np.nan, np.nan]]
    
    df = pd.DataFrame(columns=standardized_cols, data=standardized_data)
    return df

In [35]:
def get_standardized_df(standardized, state_name) -> pd.DataFrame:
    try:
        return standardized[state_name]
    except:
        standardized[state_name] = generate_empty_standardized_df(state_name)
        return standardized[state_name]

In [36]:
def drop_unkeyed_cols(df: pd.DataFrame, keyed: List[str]
        ) -> pd.DataFrame:
    for col in df.columns:
        if col not in keyed:
            return drop_unkeyed_cols(df.drop(columns=[col]), keyed)
    return df


# renames columns with names consistent with key
def rename_raw_cols(df, election_key) -> pd.DataFrame:
    new_cols = {}
    
    df.columns = [str(x) for x in df.columns]
    for name in df.columns:
        alpha_cols = re.findall('[a-zA-Z0-9]+', name)

        for col in alpha_cols:
            if col in candidate_keys or col in party_keys:
                new_cols[name] = col
                break

    new_df = df.rename(columns=new_cols)
    keyed_cols = [new_cols[col] for col in new_cols]
    
    new_df = drop_unkeyed_cols(new_df, keyed_cols)
    
    return new_df

In [37]:
def next_row_as_columns(df) -> pd.DataFrame:
    df.columns = df.iloc[0].tolist()
    return df.drop(df.index[0])

def get_state_votes(df):
    try:
        return df[['PARTY', 'VOTES']]
    except:
        pass
    try:
        return df[['PARTY.1', 'VOTES']]
    except:
        pass
    try:
        return df[['CANDIDATE', 'VOTES']]
    except:
        pass
    try:
        return df[['PARTY', 'POPULAR VOTE']]
    except:
        pass
    try:
        return df[['CANDIDATE', 'POPULAR VOTE']]
    except:
        pass
    try:
        return get_state_votes(next_row_as_columns(df))
    except:
        raise RuntimeError('Unable to get state votes')

In [45]:
def keyify_df(df: pd.DataFrame) -> pd.DataFrame:
    col_renames = {}
    for col in df.columns:
        try:
            col_renames[col] = party_key_from_candidate[col]
        except:
            col_renames[col] = party_key_from_party[col]
            
    return df.rename(columns=col_renames)

In [46]:
def standardize_county_lvl(raw_df, state_abv, election_key
        ) -> pd.DataFrame:
    df = raw_df.copy()
    
    if state_abv == 'IN':
        df = df.drop(df.index[0])
    
    df = df.drop(columns=[df.columns[0]])
    df = drop_cols(df, list(df.columns), col_is_empty)
    df = drop_cols(df, list(df.columns), col_has_percent)
    df = rename_raw_cols(df, election_key)
    
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            raise RuntimeError("Unable to cast to number")
    
    df = keyify_df(df)
    
    col_sums = dq.sum_column_values(df, list(df.columns))
    cols = [c for c, _ in col_sums]
    sums = [s for _, s in col_sums]
    
    return pd.DataFrame(data=[sums], columns=cols)

In [47]:
def standardize_state_lvl(raw_df, state_abv, election_key
        ) -> pd.DataFrame:
    df = raw_df.copy()
    df.columns = [str(x) for x in df.columns]
    
    df = drop_cols(df, list(df.columns), col_is_empty)
    df.columns = [str(x) for x in df.columns]
    df = drop_cols(df, list(df.columns), col_has_percent)
    df.columns = [str(x) for x in df.columns]
    
    df = get_state_votes(df)
    df.columns = [str(x) for x in df.columns]
          
    df = df.transpose()
    df.columns = [str(x) for x in df.columns]
    df = next_row_as_columns(df) # remove transposed numerical index
    df.columns = [str(x) for x in df.columns]
    
    df = drop_cols(df, list(df.columns), col_has_change, threshold=0.0)
    df.columns = [str(x) for x in df.columns]
    df = rename_raw_cols(df, election_key)
    
    # duplicate parties (col labels) 
    # -- write-ins or candidates in same party (e.g. senate races)
    if election_key == 'PRES16':
        if state_abv == 'WA':
            df = df.iloc[:, :-1]
        elif state_abv == 'TX':
            df = df.iloc[:, :-2]
        elif state_abv == 'VT':
            df = df.iloc[:, :-1]
        elif state_abv == 'HI':
            df = df.iloc[:, :-1]
            
    elif election_key == 'SEN16':
        if state_abv == 'AZ':
            df = df.iloc[:, :-3]
        if state_abv == 'CA':
            df = df.iloc[:, :-1]
            
    elif election_key == 'SEN18':
        if state_abv == 'CA':
            df = df.iloc[:, :-1]

    for col in df.columns:
        if not isinstance(list(df[col])[0], int):
            try:
                df[col] = pd.to_numeric(df[col])
            except:
                df = df.drop(columns=[col])
    
    df = keyify_df(df)
    
    col_sums = dq.sum_column_values(df, list(df.columns))
    cols = [c for c, _ in col_sums]
    sums = [s for _, s in col_sums]
    
    return pd.DataFrame(data=[sums], columns=cols)

In [48]:
def standardize_wiki_df(standardized_df, raw_df, 
                        state_name, state_abv, election_key
        ) -> pd.DataFrame:
    # county-level wiki election data
    if (election_key.startswith('PRES') and 
        (state_abv == 'IN' or state_abv == 'KY' or 
         state_abv == 'MT')):
        df = standardize_county_lvl(raw_df, state_abv, 
                                    election_key)
        
    # state-level wiki election data
    else:
        df = standardize_state_lvl(raw_df, state_abv,
                                   election_key)

    # TODO -- put data into standardized
    
    return standardized_df

In [49]:
""" 
Format for each standardized DataFrame:

STATE   | PRES16D | PRES16G | PRES16L | PRES16R | SEN16D | SEN16G | ... | USH18R
---------------------------------------------------------------------------------
ALABAMA | ...

"""

standardized_wiki_dfs = {}

for wiki_key in wiki_dfs:
    election_key, state_key = get_key_tuple(wiki_key)
    state_name, state_abv = get_state_tuple(state_key)
    
    raw_df = wiki_dfs[wiki_key]
    standardized_df = get_standardized_df(standardized_wiki_dfs, state_name)
    
    if (election_key.startswith('PRES') or 
        election_key.startswith('SEN') or
        election_key.startswith('USH')):
        standardized_wiki_dfs[state_name] = \
            standardize_wiki_df(standardized_df, raw_df,
                                state_name, state_abv, 
                                election_key)
    else:
        print('Election not currently used:', election_key)

         R        D      L
0  1479471  1380335  87531
         D        R
0  3755489  1730439
        R       D      L
0  661984  400602  43866
        D       R      L
0  253876  235963  14545
        R       D      L     G
0  409055  189765  28331  8496
         D        R       L      G
0  1257708  2405942  107504  27785
        D       R       L
0  376998  212813  107201
         D       R      G
0  1659907  972557  89970
         D       R
0  1633371  979210
        D       R      L
0  354649  353632  12597
         D        R      L      G
0  1370710  1215318  99277  36805
        R       D
0  237163  132810
         D        R
0  2355923  2053963
        R       D
0  536191  347816
        R      D     L
0  136210  61227  5658
        R       D
0  760241  301860
         R        D       L      G
0  4617886  4504975  207043  64399
         R        D       L      G
0  2970733  2926441  146715  49941
        D       R      L     G
0  252525  180543  14746  6220
         D       R

In [42]:
""" 
Format for concatenated standardized DataFrame:

STATE   | PRES16D | PRES16G | PRES16L | PRES16R | SEN16D | SEN16G | ... | USH18R
---------------------------------------------------------------------------------
ALABAMA | ...
ALASKA  | ...
...
WYOMING | ...

"""

wiki_states_df = pd.DataFrame()
for key in standardized_wiki_dfs:
    wiki_states_df = pd.concat([wiki_states_df, standardized_wiki_dfs[key]])

Step 7. Save processed Wikipedia data locally
------------------------------------------------------

In [43]:
# wiki_df_outfile = os.path.join('wiki', 'wiki_states.csv')

# wiki_et = et.ExtractTable(wiki_states_df, column='STATE', 
#                 outfile=wiki_df_outfile).extract_to_file()